1. $0^{th}$ level policy model $\pi^{(0)}(s_{t}|s_{t^{(1)}}^{(1)} \circ a_{t}^{(1)})$
2. $1^{th}$ level policy model $\pi^{(1)}()$

In [4]:
from model import sandwich_embedding as se 
import torch 
from model import GPTConfig

config = GPTConfig(vocab_size=50304, n_layer=12, n_head=6, n_embd=768, K=3, L=2, device="cpu", _compile=False)
B, S, D = 3, 6, config.n_embd 
K = config.K
L = config.L

# Sandwich embedding ensemble (temporal predicted token embeddings ensemble)

token_embeddings = torch.randn(B, S, D)
high_level_embeddings = torch.randn(B, S//K, D)
low_level_embeddings = torch.randn(B, S*K, D)

# (I). Embedding Ensemble
# v1. pure additive ensemble across abstraction levels
se(low_level_embeddings, token_embeddings, high_level_embeddings, K)

# (II). Conditional GPT
from model import CondGPT, GPTConfig

condgpt = CondGPT(config)
idx = torch.randint(0, 50304, (B, S))
condgpt.forward(idx, high_level_embeddings, low_level_embeddings)
condgpt.inference(idx, high_level_embeddings, low_level_embeddings)[1]

tensor([0, 0, 0])

tensor([0, 0, 0])

In [6]:
seq_len = token_embeddings.shape[1]
L2 = low_level_embeddings.shape[1]
assert seq_len*K <= L2, f"Planning without grounding is not allowed: {seq_len} > {L2}"
token_embeddings += low_level_embeddings[:, 0:seq_len*K:K]

if high_level_embeddings is not None: 
    L1 = high_level_embeddings.shape[1]
    assert L1 * K <= seq_len < (L1 + 1) * K, f"Execution without purpose or planning without grounding is not allowed: {L1 * K} < {seq_len} <= {(L1 + 1) * K}"
    cond_embeddings = high_level_embeddings.repeat_interleave(K, dim=1)
    token_embeddings[:, :L1 * K] += cond_embeddings

1. Better to put a $<begin>$ token at each abstract level, just to avoid forward propagation without token at abstract level. 